In [119]:
chat = ''
with open('chat1.txt', mode = 'r', encoding = 'utf-8') as f:
    chat = f.readlines()

In [120]:
senders = set()          # To mark whether a message traversed is from a new sender or old
messages = dict()    # Store all messages in structured format
dates = dict()       # Store all dates in structures format
last = None

for msg in chat[1:]:
    if not (msg[2] == '/' and msg[5] == '/' and msg[8] == ',' and msg[11] == ':'):
        # This means there was a newline in chat and is a part of previous message
            messages[last][-1] += ' ' + content
            continue
            
        # Extract different components of raw data
        d, t = msg.index(','), msg.index('-')
        s = t + msg[t:].index(':')
        date = msg[:d]
        tim = msg[d+2:t]
        sender = msg[t+2:s]
        content = msg[s+2:-1]
        
        # Adding data
        if sender not in senders:
            senders.add(sender)
            messages[sender] = [content]
            dates[sender] = dict()
        else:
            messages[sender].append(content)
            
        if date in dates[sender]:
            dates[sender][date] += 1
        else:
            dates[sender][date] = 1
            
        last = sender

In [121]:
print(senders)

{'Noopur Di', 'ीरे मरने लगते हैं, अगर आप', 'ell switch with a cleaning fluid ', 'ेसन ', 'Papaji', 'Jijaji', 'Mrudul', 'ीरे मरने लगते हैं, जब आप', 'Namrata Di', 'दाल ', 'छोले ', 'ु', 'Mummy', 'Jiju', 'कौरव के '}


In [113]:
def sort_dates(dates):
    dates.sort(key = lambda date: list(map(int, date.split('/')))[::-1])
    return dates

In [114]:
def get_date_difference(date1, date2):
    from datetime import date
    d1, m1, y1 = map(int, date1.split('/'))
    d2, m2, y2 = map(int, date2.split('/'))
    diff = date(y2, m2, d2) - date(y1, m1, d1)
    return abs(diff.days)

In [115]:
# Date statistics

def biggest_date_difference(dates):       # Input: list[key list of dates dictionary]
    dates = sort_dates(dates)
    if len(dates) == 1:
        return 0
    max_diff = get_date_difference(dates[0], dates[1])
    date1 = dates[0]
    date2 = dates[1]
    for i in range(1, len(dates)):
        diff = get_date_difference(dates[i-1], dates[i])
        if diff > max_diff:
            max_diff = diff
            date1 = dates[i-1]
            date2 = dates[i]
    
    return max_diff, date1, date2

def longest_streak(dates):                # Input: list[key list of dates dictionary]
    dates = sort_dates(dates)
    streak = 1
    max_streak = 1
    st, max_st = dates[0], dates[0]
    en, max_en = dates[0], dates[0]
    for i in range(1,len(dates)):
        if get_date_difference(dates[i], dates[i-1]) == 1:
            streak += 1
            en = dates[i]
        else:
            if streak > max_streak:
                max_strek = streak
                max_st = st
                max_en = en
            streak = 1
            st = dates[i]
    if streak > max_streak:
        max_strek = streak
        max_st = st
        max_en = en
    return max_streak, max_st, max_en

def most_messages_per_day(dates):              # Input: dictionary[dates dictionary]
    top3 = [[None, 0], [None, 0], [None, 0]]       # [0]: 3rd rank, [1]: 2nd rank [2]: 3rd rank
    for i in dates:
        if top3[0][1] < dates[i]:
            top3[0][0] = i
            top3[0][1] = dates[i]
        top3.sort(key = lambda x: x[1])
    return top3

In [116]:
# Message Counts

def sender_count(sender_msg):
    return len(sender_msg)

In [117]:
# Creating Dashboard
stat_sender = []
total_msgs = 0
print("Sender-wise Statistics:-")
for sender in senders:
    print("********************", sender, "********************")
    
    date_diff, st, en = biggest_date_difference(list(dates[sender].keys()))
    print("Biggest Date Difference:", date_diff, "-> From", st, "to", en)
    print()
    
    streak, st, en = longest_streak(list(dates[sender].keys()))
    print("Longest Streak of messages:", streak, "-> From", st, "to", en)
    print()
    
    top3 = most_messages_per_day(dates[sender])[::-1]
    print("Top 3 most active days:-")
    for day, count in top3:
        print(day, ":", count, "messages")
    print()
    
    count = sender_count(messages[sender])
    print("Total Messages:", count)
    stat_sender.append([sender, count])
    total_msgs += count
    print()
    print()

Sender-wise Statistics:-
******************** Mrudul ********************
Biggest Date Difference: 23 -> From 19/11/19 to 12/12/19

Longest Streak of messages: 1 -> From 02/05/20 to 03/05/20

Top 3 most active days:-
04/04/19 : 46 messages
13/10/19 : 27 messages
07/01/20 : 25 messages

Total Messages: 699


******************** Noopur Di ********************
Biggest Date Difference: 25 -> From 05/12/18 to 30/12/18

Longest Streak of messages: 1 -> From 02/05/20 to 03/05/20

Top 3 most active days:-
17/04/19 : 24 messages
07/07/19 : 24 messages
24/09/19 : 24 messages

Total Messages: 599




In [118]:
# Group Statistics
print("Total Messages:", total_msgs)
stat_sender.sort(key = lambda x: -x[1])
print("Most active participants:-")
ind = 1
for s,c in stat_sender:
    print(str(ind) + ".", s,":", c, "messages --->", str(int((c/total_msgs) * 100))+"% of messages")

Total Messages: 1298
Most active participants:-
1. Mrudul : 699 messages ---> 53% of messages
1. Noopur Di : 599 messages ---> 46% of messages
